In [ ]:
#
# Copyright (C) 2021 by Katja Gilly <katya@umh.es>
#
# This code is licensed under a Creative Commons Attribution 4.0 International License. (see LICENSE.txt for details)
#
# General Description - this notebook plots end to end time based bin delay figures per simulation set

import numpy as np, scipy.stats as st
#
def draw_box_plots (dataDelay, vms, file_name, path, slots):

  s = 0
  step = 100
  oneTimeSlot = np.zeros(vms)  # rows: cars; value: delay
  list_oneTimeSlot = np.empty((slots,0)).tolist()
  list_hundredTimeSlot = np.empty((int(slots/step),0)).tolist()

  while (s < slots):  
    stringToSearch = ", " + str(s) + ".0]"
    if not dataDelay[dataDelay.time.str.contains(stringToSearch)].empty:
      selection = dataDelay[dataDelay.time.str.contains(stringToSearch)].copy()
      how_many = selection.shape[0]
      delays = selection.avg_e2e.values
      cars = selection.car.values
      all = pd.Series(range(0,how_many-1))
      for i in all:
        oneTimeSlot[cars[i]] = delays[i]
    # Check if the array is empty
    temp = oneTimeSlot[oneTimeSlot > 0]
    if temp.size == 0:
      list_oneTimeSlot[s].append(0)
    else: 
      list_oneTimeSlot[s]= temp.tolist()
    if (int(s%step) == 0) and (s>0):
      indice = int(s/step)-1
      for a in np.arange(s-step,s,1):
        list_hundredTimeSlot[indice].extend(list_oneTimeSlot[a])
      fig, ax = plt.subplots(figsize=(5, 3))
      plt.boxplot(list_hundredTimeSlot[0:indice+1], showfliers=False)
      ax.annotate("Time x 100s", xy=(0, 10), xycoords='figure pixels')
      plt.show() 
    elif (s == slots-1): 
      indice = int(slots/step)-1
      for a in np.arange(s+1-step,s,1):
        list_hundredTimeSlot[indice].extend(list_oneTimeSlot[a])
      fig, ax = plt.subplots(figsize=(5, 3))
      plt.boxplot(list_hundredTimeSlot[0:indice+1], showfliers=False)
      ax.annotate("Time x 100s", xy=(0, 10), xycoords='figure pixels')
      plt.show() 
    s=s+1
 
  plt.clf()
  plt.figure(dpi=300)
  fig, ax = plt.subplots(figsize=(7, 5))
  plt.ylabel("delay (ms)")
  plt.boxplot(list_hundredTimeSlot, showfliers=False)
  plt.title("End to end delay per slot time")
  ax.annotate("Time x 100s", xy=(0, 10), xycoords='figure pixels')

  plt.savefig(path_images + "bin_delays_" + file_name + '.png',dpi=300)

  return list_hundredTimeSlot

time: 92.7 ms (started: 2021-09-07 05:55:38 +00:00)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

path_Delays = "/content/drive/MyDrive/processed_logs/cloudsim_output/1s_bins_3600s/"
path_images = "/content/drive/MyDrive/processed_logs/cloudsim_output/images/"

# Configure in case of directories in input files 
dirs = ['r2_e2eDelay_49xx','r2_e2eDelay_57xx','r2_e2eDelay_69xx','r2_e2eDelay_85xx']

# define vehicles and scenarios
scenarios = ["45 1 sc sc-mad","63 1 sc sc-mad","81 1 sc sc-mad","99 1 sc sc-mad","117 1 sc sc-mad",
             "45 0 sc sc-mad","63 0 sc sc-mad","81 0 sc sc-mad","99 0 sc sc-mad","117 0 sc sc-mad"]
# simulations of 3600s
vehicles = ['4928','4951','4955','5712','5734','5749','6900','6908','6923','8589','8595','8630']
VMs = ['1800','1805','1788','2072','2046','2089','2509','2493','2498','3184','3133','3185']
# simulations of 10000s
edge_hosts = ("45","63","81","99","117")
car_ranges = ("49xx","57xx","69xx","85xx")
duration = "3600"
time_slots = 1  # 1 second bins
slots = int(int(duration)/time_slots)
# Defining arrays:
# 1st dim: vehicles 
# 2nd dim: delays (access, network, e2e)
# 3rd dim: hosts and type (5x2)
avgDelay_e2e = np.zeros((12,3,10))
upper_confInf_e2e = np.zeros((12,3,10))
lower_confInt_e2e = np.zeros((12,3,10))
sim = 0
total_sims = len(scenarios) * len(vehicles)
print("total_sims =", total_sims )

# Careful, division must be equal to "steps" in fuction "draw_box_plots"
seeds_delay_slots = np.empty((int(slots/100),0)).tolist()

for scen in scenarios:
  for vehi in vehicles:
    veh_ind = vehicles.index(vehi)
    sce_ind = scenarios.index(scen)

    group = math.floor(veh_ind/3)  # group of seeds of the same scenario
    if veh_ind%3 == 0 :
      # initialise the variable for the new seed
      seeds_delay_slots = np.empty((int(slots/100),0)).tolist()
    print("group:", group, " -> ", dirs[group])
    scen_split = scen.split()
    file_name  = "e2eDelay_" + scen_split[0] + "_" + scen_split[1] + "_sc_sc-mad_" + str(VMs[veh_ind]) + "_" + str(vehi) + "_bin.csv"
    dataDelay = pd.read_csv(path_Delays + file_name, sep=',', header=0)
    
    print("index vehicles:", veh_ind, " -> ", vehi)
    print("index scenarios", sce_ind%5, " -> ", scen)
  
    # individual e2e figures and data collection
    delay_slots = draw_box_plots (dataDelay, int(VMs[veh_ind]), file_name, path_images, int(slots))
    sim = sim +1 
    #
    for a in np.arange(0,int(slots/100),1):
        seeds_delay_slots[a].extend(delay_slots[a])

    print("seeds_delay_slots:",seeds_delay_slots)
    
    if veh_ind%3 == 2 :
      plt.clf()
      plt.figure(dpi=300)
      fig, ax = plt.subplots(figsize=(7, 5))
      plt.ylabel("delay (ms)")
      plt.boxplot(seeds_delay_slots, showfliers=False)
      plt.title("End to end delay per slot time")
      ax.annotate("Time x 100s", xy=(0, 10), xycoords='figure pixels')
      plt.savefig(path_images + "bin_delays_" + vehi + "_seed_" + file_name + '.png',dpi=300)   